In [23]:
from Bio import SeqIO
import numpy as np
temp_sequences = []
for record in SeqIO.parse("/home/samridhi/boatgang/sequences.fa", "fasta"):
    temp_sequences.append(str(record.seq))

In [24]:
import pandas as pd

In [25]:
df = pd.read_csv("/home/samridhi/boatgang/labels.txt", sep="\t")

In [26]:
print temp_sequences[0]
print type(temp_sequences[0])

GGGCGATATTCACTGCTGGGACACATACTTGAAGTCAATCTCTGACTGAACAGATGGTCGGGAACTCCTTCACTCATACGATTTTAAAATCTAAGCAATCCTGTAGTTTGCGTACCTTCTCAAACCCGGAAGTGTTGACCGGTTCGTTGACGATAGAAATAAAAGCCTTGAATCTTTTTTTACATTAATTCAACCTAGTC
<type 'str'>


In [27]:
temp_target = df['task1']
temp_sequences = pd.DataFrame(temp_sequences)
temp_sequences = temp_sequences.join(temp_target)
temp_sequences.to_csv("/home/samridhi/mycsv.csv")
temp_sequences.head()

,0,task1
0,GGGCGATATTCACTGCTGGGACACATACTTGAAGTCAATCTCTGAC...,1
1,CCTAGGTTATATGATGAGTCACCTCATTTACTTAAACCCGGAAGTG...,1
2,CGGAGTGCGTTCTGAACAGATGGTCGGTTGCAGGTAACATAACCAC...,1
3,ATAGCCCTCGATGCTCTAGCATTAACCCGGAAGTGATACCCAACGC...,1
4,TTCCGCCAGAGATGTCCTAATTTCGTGAGAGCTCATTAACCCGGAA...,1


In [21]:
print temp_sequences[0]

0        GGGCGATATTCACTGCTGGGACACATACTTGAAGTCAATCTCTGAC...
1        CCTAGGTTATATGATGAGTCACCTCATTTACTTAAACCCGGAAGTG...
2        CGGAGTGCGTTCTGAACAGATGGTCGGTTGCAGGTAACATAACCAC...
3        ATAGCCCTCGATGCTCTAGCATTAACCCGGAAGTGATACCCAACGC...
4        TTCCGCCAGAGATGTCCTAATTTCGTGAGAGCTCATTAACCCGGAA...
5        TTTTTGATGAGTCACCGATGACCGCGACAGTTAGTTCACCTGCAAA...
6        TACTGTCAGTTATTGACGTCCATTTCATATTCTACAAGCTGAACAG...
7        TTAACCACTTTGATCCGTTTATACGGAGGCTGAAGGACATCTGAAC...
8        TCCCCGCATGATTTATGATGTCATGAGTCACCTGGTTTTCACTGCA...
9        ATAGGAAAGGTACACACCTCGAAAACAAACCCGGAAGTGGCCCAAC...
10       AGGTGCCAGGAATGTCTCAACCCGGAAGTGCATGGCTCTTTGCGTG...
11       GACATCACTATAACGTACATAGCCTGTTTCACTGAACAGATGGTCG...
12       GGACAATGGATCGGGTCGCAACCCGGAAGTGCATCCATGGGATTTC...
13       GCGTAAATATGAGTCACCGAACCCGATGTGTAACACTGATATAGAA...
14       GCTCCAACTCCCTCCCGGCAAGGTGGATGGGTCCTAATTCACCCGC...
15       AATTTGAAGCACTTACAACGCAGAGTCCAAGCGTCAGCAACTTCCG...
16       ACAGCCGAGCACATTCTTTTCATGAGATCACCGTACCTAACCTCCA.

In [7]:
dict_i = {"A" : 0, "C": 1, "G": 2, "T" : 3}

In [8]:
def one_hot_encode(s):
    i=0
    ohe = np.zeros((len(s), 4))
    for k in s:
        ohe[i,dict_i[k]] = 1
        i+=1
    return ohe

In [9]:
ohe_sequences = np.array([one_hot_encode(s) for s in temp_sequences])
ohe_sequences.shape

(60000, 200, 4)

In [10]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(ohe_sequences, temp_target, test_size = 0.25, random_state = 42, shuffle=True)

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Dropout,Flatten
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [12]:
model = Sequential()
model.add(Conv1D(filters=40, kernel_size=12, activation='relu', input_shape=(200,4)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 189, 40)           1960      
Total params: 1,960
Trainable params: 1,960
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.add(MaxPooling1D(pool_size=189, strides=1))

In [14]:
model.add(Dense(200,  activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())

In [15]:
model.add(Dense(1, activation = "sigmoid"))

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 189, 40)           1960      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 40)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 200)            8200      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 200)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 10,361
Trainable params: 10,361
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(loss='binary_crossentropy', optimizer='Adam',metrics=['accuracy'])

In [ ]:
model.fit(train_x, train_y, epochs=10, verbose=1)

Epoch 1/10


In [ ]:
scores = model.evaluate(test_x, test_y)
print ("Test loss ", scores[0])
print ("Test acc ", scores[1])